# permits-data / Load Data

ETL pipeline for construction permits data in Los Angeles, California, USA. This notebook documents scripts which update metadata (column names, datatypes etc.) for a PostgreSQL database. It does not actually load the data, this is done using the COPY method within PostgreSQL.

For more information:<br>
https://data.lacity.org/A-Prosperous-City/Building-and-Safety-Permit-Information/yv23-pmwf

This notebook documents the development of the pipeline which can be run through the Makefile:
```bash
set -o allexport; source .env; set +o allexport; \
make data
```
Visit the README for more information.

## Setup

In [1]:
import os
import sys

# Set path for modules
sys.path[0] = '../'

from dotenv import load_dotenv, find_dotenv
import numpy as np
import pandas as pd
import psycopg2

In [2]:
# Set notebook display options
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

In [3]:
# Get project root directory
root_dir = os.path.dirname(os.getcwd())

# Set environment variables
load_dotenv(find_dotenv());
POSTGRES_USER = os.getenv("POSTGRES_USER")
POSTGRES_PASSWORD = os.getenv("POSTGRES_PASSWORD")
POSTGRES_DB = os.getenv("POSTGRES_DB")
DB_PORT = os.getenv("DB_PORT")
DB_HOST = os.getenv("DB_HOST")
DATA_URL = os.getenv("DATA_URL")

# Environment variables specific to notebook
DATA_DIR = os.path.dirname(root_dir) + '/data'
DB_TABLE = "permits_raw"

## 1. Update PostgreSQL Metadata

In [4]:
# Connect to PostgreSQL, useful only for notebook
def connect_db():
    try:
        con = psycopg2.connect(dbname=POSTGRES_DB,
                               user=POSTGRES_USER,
                               password=POSTGRES_PASSWORD,
                                host=DB_HOST, 
                                port=DB_PORT,
                              connect_timeout=3)
        print('Connected as user "{}" to database "{}" on {}:{}\n'.format(POSTGRES_USER,POSTGRES_DB,
                                                           DB_HOST,DB_PORT))
              
    except Exception as e:
        print('Error:\n', e)
    
    return con

In [5]:
conn = connect_db()

Connected as user "postgres" to database "permits" on localhost:5432



### 1.1 Update Table Columns in PostgreSQL Database

In [6]:
# Get raw data column names
def get_table_names(table, con):
    sql = "SELECT * FROM INFORMATION_SCHEMA.COLUMNS WHERE TABLE_NAME = N'{}'".format(table)
    etl = pd.read_sql_query(sql, con)
    columns = etl['column_name']
    
    return columns

In [7]:
# Check table names
get_table_names("permits_raw", conn).head(10)

0                 assessor_book
1                 assessor_page
2               assessor_parcel
3                         tract
4                         block
5                           lot
6    reference_no_old_permit_no
7                pcis_permit_no
8                        status
9                   status_date
Name: column_name, dtype: object

In [8]:
# Retrieve table column names
old_columns = get_table_names("permits_raw", conn)

In [9]:
# Map of character replacements
replace_map = {' ': '_', '-': '_', '#': 'No', '/': '_', 
               '.': '', '(': '', ')': '', "'": ''}

# Rename columns, will update table later
def format_names(series, char_map):
    
    def replace_chars(text):
        for oldchar, newchar in char_map.items():
            text = text.replace(oldchar, newchar).lower()
        return text

    return series.apply(replace_chars)

In [10]:
# Transform table column names for permits_raw
new_columns = format_names(old_columns, replace_map)

In [11]:
new_columns.head()

0      assessor_book
1      assessor_page
2    assessor_parcel
3              tract
4              block
Name: column_name, dtype: object

In [12]:
# Creates a SQL query to update table columns and writes to text file
### add path string
def create_query(old_columns, new_columns, db_table, con, path, run=False):
    
    sql = 'ALTER TABLE {} '.format(db_table) + 'RENAME "{old_name}" to {new_name};'
    
    
    sql_query = []

    for idx, name in old_columns.iteritems():
        sql_query.append(sql.format(old_name=name, new_name=new_columns[idx]))
        
    update_names = '\n'.join(sql_query)
    
    # replace with path
    with open(path, 'w') as text:
        text.write(update_names)
        
    # Update db is desired
    if run:
        try:
            cur = con.cursor()
            print("Reading...")
            sql_file = open(path, 'r')
            print("Executing...")
            cur.execute(sql_file.read())
            con.commit()
            print("Closing connection...")
            #conn.close()
            print("Done.")
        except Exception as e:
            conn.rollback()
            print('Error:\n', e)

In [13]:
# Set path for sql query file
sql_path = root_dir + '/postgres/sql/update_names.sql'

# Connect
conn = connect_db()

# Create SQL query for permits_raw
create_query(old_columns, new_columns, run=True, con=conn, path=sql_path, db_table=DB_TABLE)

Connected as user "postgres" to database "permits" on localhost:5432

Reading...
Executing...
Error:
 column "assessor_book" of relation "permits_raw" already exists



In [14]:
# Check table names are updated
get_table_names("permits_raw", conn).head()

0      assessor_book
1      assessor_page
2    assessor_parcel
3              tract
4              block
Name: column_name, dtype: object

In [15]:
# TEST: 
assert (get_table_names("permits_raw", 
                        conn) == new_columns).mean() == 1, "Database table names do not match new table names"

In [16]:
# Extract full dataset
sql_all = 'SELECT * FROM {} LIMIT 1500;'.format(DB_TABLE)

# Extract full dataset
data = pd.read_sql_query(sql_all, conn)
data.head()

,assessor_book,assessor_page,assessor_parcel,tract,block,lot,reference_no_old_permit_no,pcis_permit_no,status,status_date,permit_type,permit_sub_type,permit_category,project_number,event_code,initiating_office,issue_date,address_start,address_fraction_start,address_end,address_fraction_end,street_direction,street_name,street_suffix,suffix_direction,unit_range_start,unit_range_end,zip_code,work_description,valuation,floor_area_la_zoning_code_definition,no_of_residential_dwelling_units,no_of_accessory_dwelling_units,no_of_stories,contractors_business_name,contractor_address,contractor_city,contractor_state,license_type,license_no,principal_first_name,principal_middle_name,principal_last_name,license_expiration_date,applicant_first_name,applicant_last_name,applicant_business_name,applicant_address_1,applicant_address_2,applicant_address_3,zone,occupancy,floor_area_la_building_code_definition,census_tract,council_district,latitude_longitude,applicant_relationship,existing_code,proposed_code
0,4317,003,***,TR 30210-C,None,LT 1,None,15044-90000-08405,Permit Finaled,09/10/2015,HVAC,1 or 2 Family Dwelling,No Plan Check,None,None,INTERNET,2015-08-18,1823,1/2,1823,1/2,S,THAYER,AVE,None,None,None,90025,None,None,None,None,None,None,CONDITIONED AIRE MECHANICAL & ENGINEERING INC,18650 PARTHENIA STREET,NORTHRIDGE,CA,C20,532440,BRETT,MOORE,HOFFER,2016-06-30,BRETT,HOFFER,None,18650 PARTHENIA ST,None,"NORTHRIDGE, CA",R3-1-O,None,0,2671.00,5,"(34.05474, -118.42628)",Net Applicant,None,None
1,5005,010,017,CHESTERFIELD SQUARE,None,465,16SL57806,16016-70000-02464,Permit Finaled,08/01/2017,Bldg-Alter/Repair,1 or 2 Family Dwelling,No Plan Check,None,None,SOUTH LA,2016-02-04,2122,None,2122,None,W,54TH,ST,None,None,None,90062,General rehabilitation for single family dwell...,40000.00,None,None,None,None,OWNER-BUILDER,None,None,None,NA,0,JAVIER,None,TALAMANTES,None,JAVIER,TALAMANTES,OWNER-BUILDER,None,None,None,C2-1VL,None,None,2325.00,8,"(33.99307, -118.31668)",Owner-Bldr,1,None
2,5154,023,022,SUN-SET TRACT,D,13,14VN81535,14016-20000-13092,Issued,08/13/2014,Bldg-Alter/Repair,Apartment,Plan Check,None,None,VAN NUYS,2014-08-13,415,None,415,None,S,BURLINGTON,AVE,None,1-30,1-30,90057,PHOTOVOLTAIC SOLAR PANELS ON ROOF OF (E) APT BLDG,37000.00,None,None,None,None,PERMACITY CONSTRUCTION CORP,5570 W WASHINGTON BLVD,LOS ANGELES,CA,B,827864,JONATHAN,SAUL,PORT,2015-11-30,LINDA,MARTON,None,710 WILSHIRE BLVD,None,"SANTA MONICA, CA",R4-1,None,None,2089.04,1,"(34.06012, -118.26997)",Agent for Owner,5,None
3,4404,030,010,TR 12086,None,2,None,16044-30000-09658,Permit Finaled,08/29/2016,HVAC,1 or 2 Family Dwelling,No Plan Check,None,None,WEST LA,2016-08-22,315,None,315,None,S,OCEANO,DR,None,None,None,90049,None,None,None,None,None,None,E/C HEATING AND AIR CONDITION,26888 CUATRO MILPAS ST,VALENCIA,CA,C20,651051,EDY,RUDOLFO,CORDON,2018-07-31,None,None,None,None,None,None,RS-1,None,0,2640.00,11,"(34.05707, -118.4732)",Contractor,None,None
4,2646,019,011,TR 7158,None,11,None,17042-90000-31792,Permit Finaled,12/28/2017,Plumbing,1 or 2 Family Dwelling,No Plan Check,None,None,INTERNET,2017-12-26,13640,None,13640,None,W,PIERCE,ST,None,None,None,91331,None,None,None,None,None,None,TITANIUM POWER INC,1545 S LA CIENEGA BLVD,LOS ANGELES,CA,B,989217,DENNIS,HARUO,MIYAHIRA,2017-12-31,YONI,GHERMEZI,None,1545 S LA CIENEGA BLVD,None,"LOS ANGELES, CA",R1-1-O,None,0,1044.03,7,"(34.25487, -118.43002)",Net Applicant,None,None


### 1.2 EDA Functions for Determining Data Types

Determining the correct data types involves some preliminary exploration of the data for which uses two helper functions: *get_snapshot* and 

In [17]:
### Returns overview with column, dtype, # unique values, # missing values and sample value
def get_snapshot(dataframe):
    
    """
    Takes an existing DataFrame and creates a pandas DataFrame 
    where each row displays the original DataFrame column name, 
    number of unique values, number of missing values, and a
    random sample value from that column. 
    
    Useful for exploring raw data to quickly figure out appropriate 
    data types.
    
    Example
    -------
    
    overview = get_overview(my_dataframe)
    
    """
    unique = dataframe.nunique(axis=0)
    is_null = dataframe.isnull().sum()
    data_types = dataframe.dtypes
    
    samples = pd.DataFrame()
    column_names = pd.DataFrame()
    
    for column, row in dataframe.iteritems():
        try:
            sample = dataframe[column].dropna(axis=0).sample()
            column_name = pd.Series(column)
        except:
            pass

        samples = pd.concat([samples, sample], axis=0).reset_index(drop=True)
        column_names = pd.concat([column_names, column_name], 
                                 axis=0, ).reset_index(drop=True)

        examples = pd.concat([column_names, samples], axis=1, ignore_index=True)
        examples.columns = ['COLUMN', 'SAMPLE VALUE']
    
    overview = pd.concat([data_types, unique, is_null], axis=1)
    overview.reset_index(inplace=True)
    overview.columns = columns=['COLUMN', 'DATA TYPE', '# UNIQUE VALUES', '# MISSING VALUES']
    overview = overview.merge(right=examples, on='COLUMN').drop_duplicates(subset=['COLUMN']).set_index('COLUMN')
    
    return overview

In [18]:
get_snapshot(data)

,DATA TYPE,# UNIQUE VALUES,# MISSING VALUES,SAMPLE VALUE
COLUMN,,,,
assessor_book,object,742,1,5071
assessor_page,object,49,1,018
assessor_parcel,object,111,1,023
tract,object,1202,5,TR 8267
block,object,96,1178,3
lot,object,296,14,10
reference_no_old_permit_no,object,520,877,13SL50790
pcis_permit_no,object,1500,0,13043-10000-03559
status,object,11,0,Issued


In [19]:
# Creates a report to show value counts for columns with less than n unique values
def explore_value_counts(dataframe, n=None, max_n=1500, columns=None, printed=True):
    
    """
    This function is helpful for quickly determining which values
    should be converted to integer or category types in a dataframe.
    
    Prints a series of custom text summaries with n value counts 
    for each column. Can work if neither n nor columns are specified.
    
    Also can return a generator yielding a text summary with n
    value counts for each column.
    
    Example:
    --------
    ## Iterates through individual tables
    gen = explore_value_counts(data, printed=False)
    print(next(gen)) 
    
    ## Prints all tables to STDOUT
    explore_value_counts(data, printed=True)
    
    Params
    --------
    dataframe : pandas DataFrame
        DataFrame with columns to be summarized.
    n : integer or string
        Max number of unique categories in column, or 'all'.
    max_n : integer
        Ceiling safeguard to avoid extremely large values of n
    columns : list 
        Columns to include in output.
    printed : bool
        If True prints to console; if False returns generator object
        which can be printed as text.
    
    Returns
    --------
    if printed=True: prints all formatted text of all tables
    
    if printed=False: generator object that outputs one table
    
    """
    
    # Parsing arguments
    if columns:
        dataframe = dataframe[columns]
    
    if n == 'all':
        n = len(dataframe) if len(dataframe) <= max_n else max_n
    elif not n:
        n = 30
    else:
        n = n if n <= max_n else max_n
        
    def make_tables():
        dataframe_n = pd.DataFrame()

        # Data selection
        for column, row in dataframe.iteritems():
            
            n_unique = dataframe[column].nunique()
            
            # Throws error if float64 is removed
            if (dataframe[column].dtype not in ['float64', '<M8[ns]']):
                dataframe_n = pd.concat([dataframe_n, dataframe[column]], axis=1)

        summary_list = []

        # Text generation
        for column, row in dataframe_n.iteritems(): 
            series = dataframe_n[column]
            name = series.name
            
            index_slice = n if n <= len(series) else len(series)
            
            # Create dataframe of value counts
            counted = series.value_counts(sort=True)[:index_slice]
            percent = series.value_counts(sort=True, normalize=True)[:index_slice]
            summary = pd.concat([counted, percent], axis=1)
            summary.columns = ['COUNT', 'PERCENTAGE']
            summary.index = summary.index.rename('UNIQUE VALUES:')
            
            # Create a custom table with n unique, missing values to print to console as text
            summary_text = 'COLUMN:   "{}"\nDATA TYPE:  {}'.format(name, series.dtype)
            summary_text = summary_text + '\nTOTAL UNIQUE:  {}'.format(name, series.nunique())
            summary_text = summary_text + '\nTOTAL MISSING:  {}'.format(series.isnull().sum())
            summary_text = summary_text + '\n' + summary.to_string() + '\n\n'

            summary_list.append(summary_text)
        
        if not printed:
            summary_gen = iter(summary_list)
            return summary_gen
        else:
            return summary_list
        
    if printed:
        print('\n'.join(make_tables()))
    else:
        return make_tables()

In [20]:
# Examine variables to determine appropriate data types
explore_value_counts(data, columns=['block'], n=10, printed=True)

COLUMN:   "block"
DATA TYPE:  object
TOTAL UNIQUE:  block
TOTAL MISSING:  1178
                COUNT  PERCENTAGE
UNIQUE VALUES:                   
2                  20    0.062112
3                  18    0.055901
1                  14    0.043478
5                  13    0.040373
4                  12    0.037267
6                  11    0.034161
7                  10    0.031056
11                  9    0.027950
C                   9    0.027950
10                  9    0.027950




## Summary

The functions in this notebook allow the pipeline to connect to a PostgreSQL database on any server, update the column names in tables, and update the data types as well. The notebook is not the pipeline itself, this can be run through the Makefile.